<a href="https://colab.research.google.com/github/SharvaniBaggani/DietGPT/blob/main/models_vision_multitask_py_%E2%80%94_Keras_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# models/vision_multitask.py
import tensorflow as tf
from tensorflow.keras import layers, Model

def build_multitask_model(input_shape=(224,224,3), backbone_name='MobileNetV2'):
    inp = layers.Input(shape=input_shape)

    # Pretrained backbone
    if backbone_name == 'MobileNetV2':
        backbone = tf.keras.applications.MobileNetV2(include_top=False, input_tensor=inp, weights='imagenet')
        x = backbone.output
        x = layers.GlobalAveragePooling2D()(x)
    else:
        backbone = tf.keras.applications.ResNet50(include_top=False, input_tensor=inp, weights='imagenet')
        x = backbone.output
        x = layers.GlobalAveragePooling2D()(x)

    # Common dense
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.3)(x)

    # BMI head - regression
    bmi_x = layers.Dense(128, activation='relu')(x)
    bmi_out = layers.Dense(1, name='bmi')(bmi_x)  # linear

    # Stress head - classification (3 classes: low, medium, high)
    stress_x = layers.Dense(128, activation='relu')(x)
    stress_out = layers.Dense(3, activation='softmax', name='stress')

    # Sleepiness - classification (3 classes)
    sleep_x = layers.Dense(128, activation='relu')(x)
    sleep_out = layers.Dense(3, activation='softmax', name='sleep')

    model = Model(inputs=inp, outputs=[bmi_out, stress_out, sleep_out])
    return model

# compile example
# model = build_multitask_model()
# losses = {'bmi': 'mse', 'stress': 'categorical_crossentropy', 'sleep': 'categorical_crossentropy'}
# model.compile(optimizer='adam', loss=losses, metrics={'bmi': 'mae', 'stress': 'accuracy', 'sleep': 'accuracy'})
